# Método del Polinomio Característico

## Implementación

In [1]:
# Importar las librearias necesarias
import numpy as np
import sympy as sp
from sympy.abc import x

# Definir el método de bisección para encontrar las raíces del polinomio característico
def bisection(char_poly, a, b, tol, max_iter):
    """
    Parámetros:
    - char_poly: El polinomio característico simbólico cuya raíz queremos encontrar.
    - a, b: El intervalo [a, b] donde se sospecha que existe la raíz.
    - tol: La tolerancia para la precisión de la raíz.
    - max_iter: El número máximo de iteraciones permitidas.

    Retorna:
    - La raíz del polinomio dentro de la tolerancia dada, o None si no se encuentra ninguna raíz.
    """

    # Evaluar el polinomio en el extremo 'a' 
    fa = float(char_poly.evalf(subs={x:a}))

    # Comprobar si a es un raíz
    if abs(fa) < 1e-6:
        return a

    # Evaluar el polinomio en el extremo 'b'
    fb = float(char_poly.evalf(subs={x:b}))
    
    # Comprobar si b es un raíz
    if abs(fb) < 1e-6:
        return b

    # Comprobar si el intervalo inicial [a, b] es válido
    if np.sign(fa) * fb > 0:
        # Ambos tienen el mismo signo, por lo que no hay raíz en [a, b]
        return None
        
    # Realizar el método de bisección por 'max_iter' iteraciones
    for i in range(max_iter):
        # Calcular el punto medio del intervalo
        p = a + (b - a) / 2
        # Evaluar el polinomio en el punto medio
        fp = float(char_poly.evalf(subs={x:p}))

        # Comprobar la convergencia
        # Si f(p) está cerca de 0 o el ancho del intervalo es menor que la tolerancia
        if (fp == 0) or ((b - a) / 2 < tol):
            return p 

        # Actualizar el intervalo según el signo de f(p)
        elif np.sign(fa) * fp < 0:
            b = p
        elif np.sign(fb) * fp < 0:
            a = p
            
    # Si se alcanza el número máximo de iteraciones sin convergencia
    return None

# Definir el método de Gauss-Seidel para encontrar los vectores propios
def gauss_seidel(A, b, x0, tol, max_iter):
    """
    Parámetros:
    - A: Matriz cuadrada de coeficientes del sistema (numpy array de tamaño n x n).
    - b: Vector de términos independientes (numpy array de tamaño n).
    - x0: Vector inicial para la solución (numpy array de tamaño n).
    - tol: Tolerancia para el criterio de convergencia relativo.
    - max_iter: Número máximo de iteraciones permitidas.

    Retorna:
    - x_new: El vector solución aproximado si el método converge dentro de las iteraciones permitidas.
    - None: Si el método no converge o si los parámetros iniciales son inválidos.
    """
    
    # Verificar si A es una matriz cuadrada
    n_A, m_A = A.shape
    if n_A != m_A:
        return None

    # Verificar si algún elemento de la diagonal es cero
    diag_A = np.diag(A)
    if np.any(np.abs(diag_A) < 1e-6):
        return None

    # Inicializar el vector actual con el valor inicial proporcionado
    x_curr = x0

     # Iterar hasta alcanzar el número máximo de iteraciones permitido
    for it in range(max_iter):
        # Crear una copia del vector actual para almacenar los nuevos valores
        x_new = np.copy(x_curr) 

        # Actualizar cada componente del vector solución
        for i in range(n_A):
            # Calcular la suma de los términos actualizados
            sum_updated = sum(A[i, :i] * x_new[:i])

            # Calcular la suma de los términos de la solución previa
            sum_previous = sum(A[i, i+1:] * x_curr[i+1:])
            
             # Calcular el nuevo valor para la i-ésima componente de la solución
            x_new[i] = (b[i] - sum_updated - sum_previous) / A[i, i] 
            
        diff_norm = np.linalg.norm(x_new - x_curr, ord=np.inf)
        new_norm = np.linalg.norm(x_new, ord=np.inf)
        
        # Comprobar el criterio de convergencia
        if new_norm < 1e-6: # Caso especial para solución cero
            if diff_norm < tol: # Verificar la tolerancia con el error absoluto
                return x_new
        else:
            if diff_norm / new_norm < tol: # Verificar el error relativo
                return x_new
                
        # Actualizar el vector solución actual con los nuevos valores calculados  
        x_curr = x_new

    # Si se alcanza el número máximo de iteraciones sin convergencia
    return None
            

## Testeo

In [2]:
n = 2
# Definir una matriz nxn 'A'
A = sp.Matrix([[2, 1],
               [3, 4]])

# Crear una matriz identidad nxn 'I'
I = sp.eye(n)

# Calcular el polinomio característico de la matriz 'A'
# char_poly = det(A - xI), donde x es el valor propio
char_poly = (A-(x*I)).det()

# Crear un conjunto vacío para almacenar las raíces encontradas
roots = set()

# Crear un arreglo de puntos equiespaciados entre -1000 y 1000 con un tamaño de paso 1
interval = np.arange(-1000, 1000, 1, dtype=float)

# Iterar sobre cada punto en el intervalo definido
for i in interval:
    # Llamar al método de bisección para encontrar una raíz en el intervalo [i, i+1]
    ans = bisection(char_poly, i, i+1, 1e-6, 10000)
    # Si se encuentra una raíz, añadirla al conjunto de raíces
    if ans != None:
        roots.add(ans)


# Crear un vector de ceros para los términos independientes 'b', con el mismo tamaño que la matriz A
b = np.zeros(n, dtype=float)

# Definir un vector inicial para el método de Gauss-Seidel
x0 = np.array([1, 1], dtype=float)

# Iterar sobre cada valor propio encontrado en 'roots'
for root in roots:
    # Imprimir el valor propio actual
    print("Valor propio " + str(root))
    
    # Calcular la matriz M como A - λI, donde λ es el valor propio actual
    M = np.array(A, dtype=float) - (root * np.identity(n, dtype=float)) 

    # Llamar al método de Gauss-Seidel para encontrar el vector propio asociado al valor propio actual
    eigenvector = gauss_seidel(M, b, x0, 1e-6, 1000)

    # Verificar si se encontró un vector propio (es decir, si Gauss-Seidel no devolvió None)
    if type(eigenvector) != type(None):
        # Imprimir el vector propio asociado al valor propio actual
        print("Vector propio " + str(eigenvector))
    print()
    

Valor propio 1.0
Vector propio [-1.  1.]

Valor propio 5.0
Vector propio [0.33333333 1.        ]



In [3]:
n = 2
A = sp.Matrix([[3, 2],
               [3, 4]])
I = sp.eye(n)
char_poly = (A-(x*I)).det()
roots = set()
interval = np.arange(-1000, 1000, 1, dtype=float)

for i in interval:
    ans = bisection(char_poly, i, i+1, 1e-6, 10000)
    if ans != None:
        roots.add(ans)
        
b = np.zeros(n, dtype=float)
x0 = np.array([1, 1], dtype=float)
for root in roots:
    print("Valor propio " + str(root))
    M = np.array(A, dtype=float) - (root * np.identity(n, dtype=float)) 
    eigenvector = gauss_seidel(M, b, x0, 1e-6, 10000)
    if type(eigenvector) != type(None):
        print("Vector propio " + str(eigenvector))
    print()
    

Valor propio 1.0
Vector propio [-1.  1.]

Valor propio 6.0
Vector propio [0.66666667 1.        ]



In [4]:
n = 2
A = sp.Matrix([[2, 3],
               [1, 4]])
I = sp.eye(n)
char_poly = (A-(x*I)).det()
roots = set()
interval = np.arange(-1000, 1000, 1, dtype=float)

for i in interval:
    ans = bisection(char_poly, i, i+1, 1e-6, 10000)
    if ans != None:
        roots.add(ans)
        
b = np.zeros(n, dtype=float)
x0 = np.array([1, 1], dtype=float)
for root in roots:
    print("Valor propio " + str(root))
    M = np.array(A, dtype=float) - (root * np.identity(n, dtype=float)) 
    eigenvector = gauss_seidel(M, b, x0, 1e-6, 10000)
    if type(eigenvector) != type(None):
        print("Vector propio " + str(eigenvector))
    print()
    

Valor propio 1.0
Vector propio [-3.  1.]

Valor propio 5.0
Vector propio [1. 1.]



In [5]:
n = 3
A = sp.Matrix([[1, 1, 2],
               [2, 1, 1],
               [1, 1, 3]])
I = sp.eye(n)
char_poly = (A-(x*I)).det()
roots = set()
interval = np.arange(-1000, 1000, 0.5, dtype=float)

for i in interval:
    ans = bisection(char_poly, i, i+1, 1e-6, 10000)
    if ans != None:
        roots.add(ans)
        
b = np.zeros(n, dtype=float)
x0 = np.array([1, 1, 1], dtype=float)
for root in roots:
    print("Valor propio " + str(root))
    M = np.array(A, dtype=float) - (root * np.identity(n, dtype=float)) 
    eigenvector = gauss_seidel(M, b, x0, 1e-4, 100)
    if type(eigenvector) != type(None):
        print("Vector propio " + str(eigenvector))
    print()
    

Valor propio -0.2851419448852539
Vector propio [-1.89361543  3.27381718 -0.42013459]

Valor propio 0.7781229019165039

Valor propio 4.507018089294434
Vector propio [0.83759102 0.78460427 1.07642721]



In [6]:
n = 3
A = sp.Matrix([[1, 1, 2],
               [2, 1, 3],
               [1, 1, 1]])
I = sp.eye(n)
char_poly = (A-(x*I)).det()
roots = set()
interval = np.arange(-1000, 1000, 0.5, dtype=float)

for i in interval:
    ans = bisection(char_poly, i, i+1, 1e-6, 10000)
    if ans != None:
        roots.add(ans)
        
b = np.zeros(n, dtype=float)
x0 = np.array([1, 1, 1], dtype=float)
for root in roots:
    print("Valor propio " + str(root))
    M = np.array(A, dtype=float) - (root * np.identity(n, dtype=float)) 
    eigenvector = gauss_seidel(M, b, x0, 1e-4, 100)
    if type(eigenvector) != type(None):
        print("Vector propio " + str(eigenvector))
    print()
    

Valor propio -0.692021369934082
Vector propio [-0.27814242 -0.77897301  0.62476482]

Valor propio -0.35689640045166016

Valor propio 4.048916816711426
Vector propio [1.08587954 1.56916762 0.87081653]



In [7]:
n = 3
A = sp.Matrix([[2, 1, 2],
               [1, 1, 3],
               [1, 1, 1]])
I = sp.eye(n)
char_poly = (A-(x*I)).det()
roots = set()
interval = np.arange(-1000, 1000, 0.5, dtype=float)

for i in interval:
    ans = bisection(char_poly, i, i+1, 1e-6, 10000)
    if ans != None:
        roots.add(ans)
        
b = np.zeros(n, dtype=float)
x0 = np.array([1, 1, 1], dtype=float)
for root in roots:
    print("Valor propio " + str(root))
    M = np.array(A, dtype=float) - (root * np.identity(n, dtype=float)) 
    eigenvector = gauss_seidel(M, b, x0, 1e-4, 100)
    if type(eigenvector) != type(None):
        print("Vector propio " + str(eigenvector))
    print()
    

Valor propio -0.7615575790405273
Vector propio [-0.18403797 -2.21008176  1.35909252]

Valor propio 0.6366720199584961

Valor propio 4.124884605407715
Vector propio [1.50192282 1.35981857 0.91579106]



In [8]:
n = 4
A = sp.Matrix([[1, 1, 1, 2],
               [2, 1, 1, 1],
               [3, 2, 1, 2],
               [2, 1, 1, 4]])
I = sp.eye(n)
char_poly = (A-(x*I)).det()
roots = set()
interval = np.arange(-1000, 1000, 0.5, dtype=float)

for i in interval:
    ans = bisection(char_poly, i, i+1, 1e-6, 10000)
    if ans != None:
        roots.add(ans)
        
b = np.zeros(n, dtype=float)
x0 = np.array([1, 1, 1, 1], dtype=float)
for root in roots:
    print("Valor propio " + str(root))
    M = np.array(A, dtype=float) - (root * np.identity(n, dtype=float)) 
    eigenvector = gauss_seidel(M, b, x0, 1e-4, 100)
    if type(eigenvector) != type(None):
        print("Vector propio " + str(eigenvector))
    print()
    

Valor propio -0.7356424331665039
Vector propio [-1.76541596  0.866607    1.85624221  0.17061734]

Valor propio -0.4074563980102539

Valor propio 1.5085630416870117

Valor propio 6.634533882141113
Vector propio [0.67782544 0.61169355 0.97436127 1.11659437]



In [9]:
n = 4
A = sp.Matrix([[1, 2, 1, 2],
               [2, 1, 1, 1],
               [3, 2, 1, 2],
               [2, 1, 1, 4]])
I = sp.eye(n)
char_poly = (A-(x*I)).det()
roots = set()
interval = np.arange(-1000, 1000, 0.5, dtype=float)

for i in interval:
    ans = bisection(char_poly, i, i+1, 1e-6, 10000)
    if ans != None:
        roots.add(ans)
        
b = np.zeros(n, dtype=float)
x0 = np.array([1, 1, 1, 1], dtype=float)
for root in roots:
    print("Valor propio " + str(root))
    M = np.array(A, dtype=float) - (root * np.identity(n, dtype=float)) 
    eigenvector = gauss_seidel(M, b, x0, 1e-4, 100)
    if type(eigenvector) != type(None):
        print("Vector propio " + str(eigenvector))
    print()

Valor propio -0.46744251251220703

Valor propio 1.7281160354614258

Valor propio 6.827261924743652
Vector propio [0.80165594 0.65287337 1.03651654 1.16462566]

Valor propio -1.087935447692871
Vector propio [-2.71896547  1.34188428  2.27930031  0.35706946]



## Comentarios
El método del polinomio característico se basa en una idea clara, facilitando su entendimiento e implementación. En base a los resultados de los casos de prueba, se puede notar que el método funciona bien para encontrar valores propios, si bien se introduce algo de error al calcular el polinomio característico, no es algo excesivo. Por otro lado, usar este método para calcular los vectores propios, resulta ser algo más complejo y susceptible a errores, pues el método no calcula directamente los vectores propios, lo que requiere resolver sistemas de ecuaciones lineales adicionales. Así mismo, puede ser computacionalmente costoso, esto se ve reflejado en los constantes errores de overflow encontrados durante el testeo. Hecho que empeora de forma considerable dependiendo del tamaño de la matriz inicial.
